# Stop Data
This file will read in the csv data from a file, 
clean and prepare it 
enter it in the stops database

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

# Read data from csv file 

In [2]:
stop_file = "static/data/Police_Stop_data2.csv"
stop_df=pd.read_csv(stop_file)
stop_df.tail(10)

C:\Users\blizz\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,OBJECTID,callDisposition,citationIssued,gender,lastUpdateDate,lat,long,masterIncidentNumber,neighborhood,personSearch,policePrecinct,preRace,problem,race,reason,responseDate,vehicleSearch,x,y
109032,109033,GOA-Gone on Arrival,NO,Unknown,1546417236000,45.012173,-93.248236,19-001065,Holland,NO,2.0,Unknown,Suspicious Person (P),Unknown,Citizen / 9-1-1,2019-01-01 18:19:07.000000,NO,-1.038035e+07,5.623438e+06
109033,109034,ADV-Advised,NO,Unknown,1546417236000,44.984623,-93.303194,19-001089,Near - North,NO,4.0,Black,Suspicious Person (P),Unknown,Citizen / 9-1-1,2019-01-01 18:51:43.000000,NO,-1.038646e+07,5.619101e+06
109034,109035,AOK- All OK,NaN,NaN,1546417236000,44.956439,-93.252414,19-001095,Midtown Phillips,NaN,3.0,NaN,Traffic Law Enforcement (P),NaN,NaN,2019-01-01 19:08:57.000000,NaN,-1.038081e+07,5.614666e+06
109035,109036,GOA-Gone on Arrival,NO,Male,1546417236000,44.904434,-93.231043,19-001097,Wenonah,NO,3.0,Unknown,Suspicious Person (P),Unknown,Citizen / 9-1-1,2019-01-01 19:10:20.000000,NO,-1.037843e+07,5.606489e+06
109036,109037,ADV-Advised,NO,Male,1546417236000,44.897229,-93.300166,19-001104,Kenny,NO,5.0,Unknown,Suspicious Vehicle (P),Latino,Citizen / 9-1-1,2019-01-01 19:31:12.000000,NO,-1.038613e+07,5.605357e+06
109037,109038,GOA-Gone on Arrival,NO,Unknown,1546417236000,45.044855,-93.311791,19-001106,Shingle Creek,NO,4.0,Unknown,Suspicious Vehicle (P),Unknown,Citizen / 9-1-1,2019-01-01 19:34:06.000000,NO,-1.038742e+07,5.628586e+06
109038,109039,ADV-Advised,NO,Female,1546417237000,45.003070,-93.261570,19-001107,Sheridan,NO,2.0,Unknown,Traffic Law Enforcement (P),Black,Equipment Violation,2019-01-01 19:36:27.000000,NO,-1.038183e+07,5.622005e+06
109039,109040,ADV-Advised,NO,Male,1546417237000,44.962730,-93.253960,19-001114,Ventura Village,NO,3.0,Unknown,Traffic Law Enforcement (P),White,Moving Violation,2019-01-01 19:47:29.000000,NO,-1.038098e+07,5.615656e+06
109040,109041,ADV-Advised,NO,Male,1546417236000,44.937771,-93.268239,19-001139,Central,NO,3.0,Latino,Suspicious Person (P),White,Citizen / 9-1-1,2019-01-01 20:48:46.000000,NO,-1.038257e+07,5.611730e+06
109041,109042,UTL-Unable to Locate,NO,Male,1546417236000,45.001102,-93.313329,19-001159,Willard - Hay,NO,4.0,White,Suspicious Person (P),Unknown,Citizen / 9-1-1,2019-01-01 21:52:06.000000,NO,-1.038759e+07,5.621695e+06


# Imports for database


In [3]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Set up tables

In [4]:
# Define our neighborhoodData table
class neighborhoodData(Base):
    __tablename__ = 'neighborhoodData'
    id = Column(Integer, primary_key=True)
    neighborhood = Column(String)
    totalPop = Column(Integer)
    margin = Column(Integer)
    stopCnt=Column(Integer)

In [5]:
# Define our stopData table
class stopData(Base):
    __tablename__ = 'stopData'
    OBJECTID = Column(Integer, primary_key=True)
    neighborhood = Column(String)
    responseDate = Column(String)
    citationIssued = Column(String)
    lat= Column(Float)
    lon= Column(Float)
    gender =Column(String)
    responseDow = Column(Integer)
    responseDay =Column(Integer)
    responseMonth = Column(Integer)
    responseMonthName =Column(String)
    responseYear = Column(Integer)
    

In [6]:
# Define dowData table
class dowData(Base):
    __tablename__ = 'dowData'
    neighborhood = Column((String), primary_key=True)
    gender =Column(String)
    responseDow = Column(Integer)
    genderCount = Column(Integer)

In [7]:
# Define our citationData table
class citationData(Base):
    __tablename__ = 'citationData'
    responseDate = Column(String)
    neighborhood = Column(String, primary_key=True)
    citationIssued = Column(String)
    responseDay =Column(Integer)
    responseMonth = Column(Integer)
    responseMonthName =Column(String)
    responseYear = Column(Integer)
    citationCnt = Column(Integer)
    

In [8]:
# tables only in python
Base.metadata.tables
# Create our database engine
engine = create_engine('sqlite:///stops.sqlite')
# This is where we create our tables in the database
Base.metadata.create_all(engine)
# The ORM’s “handle” to the database is the Session.
from sqlalchemy.orm import Session
session = Session(engine)

# Reduce Columns
Reduce the columns to those needed and eliminate rows with null gender  
check count to confirm later operations  

In [9]:
      #select needed columns
select_df=stop_df.loc[:,["OBJECTID", "responseDate","citationIssued","lat","long","gender","neighborhood"]]

# remove null gender rows
reducedA_df=select_df.dropna(axis=0,subset=['gender'])

# remove null neighborhood rows
reduced_df=reducedA_df.dropna(axis=0,subset=['neighborhood'])
reduced_df.head()

#column name change from long to lon
reduced_df=reduced_df.rename(columns={'long':'lon'})
reduced_df.tail()

,OBJECTID,responseDate,citationIssued,lat,lon,gender,neighborhood
109037,109038,2019-01-01 19:34:06.000000,NO,45.044855,-93.311791,Unknown,Shingle Creek
109038,109039,2019-01-01 19:36:27.000000,NO,45.003070,-93.261570,Female,Sheridan
109039,109040,2019-01-01 19:47:29.000000,NO,44.962730,-93.253960,Male,Ventura Village
109040,109041,2019-01-01 20:48:46.000000,NO,44.937771,-93.268239,Male,Central
109041,109042,2019-01-01 21:52:06.000000,NO,45.001102,-93.313329,Male,Willard - Hay


# Get a list of neighborhoods
remove null neighborhood

In [10]:
#Get a listing and count of neighborhood stops
neighbor_df=reduced_df.groupby('neighborhood') ["neighborhood"].count()
neighborhood_df=pd.DataFrame(neighbor_df)
neighborhood_df=neighborhood_df.rename(columns={'neighborhood':'stopCnt'})

neighborhood_df=neighborhood_df.sort_values("neighborhood")
neighborhood_df.head()

,stopCnt
neighborhood,
Armatage,158
Audubon Park,1250
Bancroft,223
Beltrami,458
Bottineau,656


# Read in population data and merge with neighborhoods

In [11]:
pop_file = "static/data/population_data.csv"
pop_df=pd.read_csv(pop_file)
pop_df.tail(10)

,neighborhood,totalPop,margin
75,Ventura Village,6028,484
76,Victory,4550,279
77,Waite Park,5558,245
78,Webber - Camden,5296,322
79,Wenonah,4464,223
80,West Calhoun,1938,146
81,Whittier,14828,781
82,Willard - Hay,9345,538
83,Windom,4844,240
84,Windom Park,5381,279


In [12]:
merge_df=pd.merge(neighborhood_df,pop_df,on="neighborhood",how="left")
merge_df=merge_df[['neighborhood','totalPop','margin','stopCnt']]
merge_df

,neighborhood,totalPop,margin,stopCnt
0,Armatage,4915.0,262.0,158
1,Audubon Park,5007.0,237.0,1250
2,Bancroft,3513.0,231.0,223
3,Beltrami,1142.0,144.0,458
4,Bottineau,1618.0,170.0,656
5,Bryant,3179.0,324.0,167
6,Bryn - Mawr,2798.0,124.0,242
7,CARAG,5723.0,262.0,970
8,Camden Industrial,NaN,NaN,74
9,Cedar - Isles - Dean,2901.0,270.0,249


In [13]:
merge_df.to_csv("static/data/neighborhood_data.csv", index=False,header=True)

# Put neighborhood data in the table

In [14]:
merge_df.to_sql(neighborhoodData.__tablename__, engine, if_exists='replace')

session.commit()

In [15]:
engine.execute('select * from neighborhoodData').fetchall()

[(0, 'Armatage', 4915.0, 262.0, 158),
 (1, 'Audubon Park', 5007.0, 237.0, 1250),
 (2, 'Bancroft', 3513.0, 231.0, 223),
 (3, 'Beltrami', 1142.0, 144.0, 458),
 (4, 'Bottineau', 1618.0, 170.0, 656),
 (5, 'Bryant', 3179.0, 324.0, 167),
 (6, 'Bryn - Mawr', 2798.0, 124.0, 242),
 (7, 'CARAG', 5723.0, 262.0, 970),
 (8, 'Camden Industrial', None, None, 74),
 (9, 'Cedar - Isles - Dean', 2901.0, 270.0, 249),
 (10, 'Cedar Riverside', 8576.0, 760.0, 1166),
 (11, 'Central', 8737.0, 451.0, 1299),
 (12, 'Cleveland', 2980.0, 213.0, 613),
 (13, 'Columbia Park', 1717.0, 91.0, 321),
 (14, 'Como', 7208.0, 590.0, 842),
 (15, 'Cooper', 3614.0, 199.0, 242),
 (16, 'Corcoran', 4239.0, 306.0, 641),
 (17, 'Diamond Lake', 5583.0, 224.0, 240),
 (18, 'Downtown East', 1711.0, 123.0, 545),
 (19, 'Downtown West', 7357.0, 482.0, 6879),
 (20, 'ECCO', 2497.0, 197.0, 525),
 (21, 'East Harriet', 3667.0, 158.0, 319),
 (22, 'East Isles', 3315.0, 222.0, 863),
 (23, 'East Phillips', 4743.0, 631.0, 2364),
 (24, 'Elliot Park', 66

# modify Gender 
combine gender Unknown and Gender Non-Conforming into gender Other

In [ ]:
#reduce warnings
pd.options.mode.chained_assignment = None  # default='warn'
#fill in Unknown vaue with Other
reduced_df.loc[reduced_df.gender=="Unknown","gender"] = "Other"
      # fill in Gender Non-Conforming values with other
reduced_df.loc[reduced_df.gender =="Gender Non-Conforming","gender"] = "Other"
reduced_df.tail()

In [ ]:
#what gender values are present ?
#verify count
gender=reduced_df.groupby(["gender"]).count()
gender

# Modify citationIssued 
change NaN to NO

In [ ]:
# change null citationIssued to NO
reduced_df[["citationIssued"]]=reduced_df[["citationIssued"]].fillna(value='NO')
reduced_df.head()

In [ ]:
# verify count
citation=reduced_df.groupby(["citationIssued"]).count()
citation

# Parse date
responseDow = day of the week  
responseDay = day of the month as zero padded decimal  
responseDonth = month as a zero padded decimal  
responseMonthName= month as abbreviated name  
responseYear  

In [ ]:
#parse date
date_list= [datetime.strptime( entry,"%Y-%m-%d %H:%M:%S.%f") for entry in reduced_df["responseDate"]]
# determine response day of week
dow_list= [datetime.strftime(entry,"%w") for entry in date_list]

# split off response day, month and year
day_list = [datetime.strftime(entry,"%d") for entry in date_list]
month_list=[datetime.strftime(entry,"%m") for entry in date_list]
month_name_list=[datetime.strftime(entry,"%b") for entry in date_list]
year_list= [datetime.strftime(entry,"%Y") for entry in date_list]

#add columns to reduced_df
reduced_df.loc[:,"responseDow"]=dow_list
reduced_df.loc[:,"responseDay"]=day_list
reduced_df.loc[:,"responseMonth"]=month_list
reduced_df.loc[:,"responseMonthName"]=month_name_list
reduced_df.loc[:,"responseYear"]=year_list
reduced_df.tail()

# Keep just 2018 data

In [ ]:
reduced_df=reduced_df.loc[reduced_df["responseYear"]== '2018']
reduced_df

# Deliver clean stop data
to file "static/data/clean_data.csv"  
or to database

In [ ]:
reduced_df.to_csv("static/data/clean_stop_data.csv", index=False,header=True)

# Add Stop data to the database

In [ ]:
reduced_df.to_sql(stopData.__tablename__, engine, if_exists='replace',chunksize=50)

session.commit()

In [ ]:
#test for data
neighbor = session.query(stopData.neighborhood, stopData.lat, stopData.lon).filter_by(neighborhood="Como").first()
neighbor

# Prep responseDow data
get count by neighborhood, responseDow and gender

In [ ]:
# get count of genders by neighborhood and responseDow
group_df=reduced_df.groupby(["neighborhood","responseDow","gender"]).count()
group_df.head()

In [ ]:
# remove indexing rename columns and reduce columns
new_group=group_df.reset_index()
next_group=new_group.rename(columns={"OBJECTID":"genderCount"})
dow_group=next_group[["neighborhood","responseDow","gender","genderCount"]]
dow_group.head()

# Deliver clean day of week data  
to file "static/data/dow_clean_data.csv" 
or database

In [ ]:
# write ad csv file -- json file doesn't come out right
dow_group.to_csv("static/data/clean_dow_data.csv" ,index=False, header=True)

# Add dow Data to database

In [ ]:
dow_group.to_sql(dowData.__tablename__, engine, if_exists='replace',chunksize=50)

session.commit()

# Prep line chart data 
graph of citations given per day of month

# Remove all citations with value 'NO'

In [ ]:
line = reduced_df[reduced_df.citationIssued == 'YES']
line.tail()

# Keep only relevant columns

In [ ]:
lines_df = line.drop(['OBJECTID', 'responseDow','lat', 'lon', 'gender'], axis=1)
lines_df.head()

# Add citation count column for graphing

In [ ]:
lines_df['citationCnt'] = lines_df.groupby('neighborhood')['citationIssued'].transform('count')
lines_df.head()

# Write out clean csv file

In [ ]:
## Clean csv file to be saved in GitHub repository
lines_df.to_csv("static/data/clean_citation_data.csv", index=False,header=True)

# Add line data to database

In [ ]:
lines_df.to_sql(citationData.__tablename__, engine, if_exists='replace',chunksize=50)

session.commit()

In [ ]:
#test for data
citation = session.query(citationData.neighborhood, citationData.citationCnt).filter_by(neighborhood="Como").first()

citation